In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install torchcam

In [ ]:
# Install the most up-to-date version from GitHub
!pip install -e git+https://github.com/frgfm/torch-cam.git#egg=torchcam

In [1]:
%matplotlib inline
# All imports
import matplotlib.pyplot as plt
import torch
from torch.nn.functional import softmax, interpolate
from torchvision.io.image import read_image
from torchvision.models import resnet18, densenet121, resnet50
from torchvision.transforms.functional import normalize, resize, to_pil_image

from torchcam.methods import SmoothGradCAMpp, LayerCAM, GradCAMpp, GradCAM, ScoreCAM
from torchcam.utils import overlay_mask

/opt/conda/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/opt/conda/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZNK3c107SymBool10guard_boolEPKcl'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
from cv2 import imread, createCLAHE # read and equalize images
from glob import glob
import matplotlib.pyplot as plt
%matplotlib inline
from pandas.core.common import flatten

import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

plt.ion()   # interactive mode

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [3]:
from __future__ import print_function, division

# pytorch imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.autograd import Variable
import torchvision
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils

# image imports
from skimage import io, transform
from PIL import Image

# general imports
import os
import time
from shutil import copyfile
from shutil import rmtree

# data science imports
import pandas as pd
import numpy as np
import csv

In [4]:
from itertools import chain
import random
np.random.seed(25)

df = pd.read_csv('../input/data/Data_Entry_2017.csv')
df.drop(['OriginalImage[Width', 'Height]', 'OriginalImagePixelSpacing[x', 'y]', 'Unnamed: 11'], axis=1, inplace=True)
num_obs = len(df)
# print('Number of observations:',num_obs)

my_glob = glob('../input/data/images*/images/*.png')
# print('Number of Observations: ', len(my_glob))

full_img_paths = {os.path.basename(x): x for x in my_glob}
df['full_path'] = df['Image Index'].map(full_img_paths.get)

train_val_list = pd.read_csv('../input/data/train_val_list.txt', header=None, names = ['image_list'])
test_list = pd.read_csv('../input/data/test_list.txt', header=None, names = ['image_list'])

train = df[df['Image Index'].isin(train_val_list['image_list'].values)].reset_index(drop=True)
test = df[df['Image Index'].isin(test_list['image_list'].values)].reset_index(drop=True)

labels_discard = ['Consolidation', 'Edema', 'Emphysema', 'Fibrosis', 'Hernia', 'Pleural_Thickening']
for i in labels_discard:
    train = train[~train['Finding Labels'].str.contains(i)]
    test = test[~test['Finding Labels'].str.contains(i)]

# train = pd.concat([train[~train['Finding Labels'].str.contains('No Finding')],
#                   train[train['Finding Labels'].str.contains('No Finding')].drop_duplicates(subset=['Finding Labels', 'Patient ID', 'View Position'], keep='first')]).sort_values(by='Image Index')

def one_hot_enc(df):
    df['Finding Labels'] = df['Finding Labels'].map(lambda x: x.replace('No Finding', ''))
    all_labels = np.unique(list(chain(*df['Finding Labels'].map(lambda x: x.split('|')).tolist())))
    for c_label in all_labels:
        if len(c_label)>1: # leave out empty labels
            df[c_label] = df['Finding Labels'].map(lambda finding: 1 if c_label in finding else 0)
    return df
        
train = one_hot_enc(train)
test = one_hot_enc(test)

train_image_paths = list(flatten(train['full_path'].values))
test_image_paths = list(flatten(test['full_path'].values))

l = np.unique(train['Patient ID'].values)
np.random.shuffle(l)
cut = int(np.round(((90/100)*len(l)), decimals=0))
train_values = l[:cut]
val_values = l[cut:]
train_dict = dict.fromkeys(train_values, 'train')
train_dict.update(dict.fromkeys(val_values, 'val'))
train['fold'] = train['Patient ID'].map(train_dict.get)

train = train[['full_path', 'Image Index', 'fold', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
               'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']]
test['fold'] = 'test'
test = test[['full_path', 'Image Index', 'fold', 'Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration',
             'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']]

In [5]:
df = train[train[['Atelectasis', 'Cardiomegaly', 'Effusion', 'Infiltration', 'Mass', 'Nodule', 'Pneumonia', 'Pneumothorax']].sum(axis=1) == 1]
df.reset_index(drop=True, inplace=True)

In [6]:
df

,full_path,Image Index,fold,Atelectasis,Cardiomegaly,Effusion,Infiltration,Mass,Nodule,Pneumonia,Pneumothorax
0,../input/data/images_001/images/00000001_000.png,00000001_000.png,train,0,1,0,0,0,0,0,0
1,../input/data/images_001/images/00000005_006.png,00000005_006.png,train,0,0,0,1,0,0,0,0
2,../input/data/images_001/images/00000008_000.png,00000008_000.png,train,0,1,0,0,0,0,0,0
3,../input/data/images_001/images/00000008_002.png,00000008_002.png,train,0,0,0,0,0,1,0,0
4,../input/data/images_001/images/00000010_000.png,00000010_000.png,train,0,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
19720,../input/data/images_012/images/00030764_000.png,00030764_000.png,train,0,1,0,0,0,0,0,0
19721,../input/data/images_012/images/00030770_001.png,00030770_001.png,train,0,0,0,1,0,0,0,0
19722,../input/data/images_012/images/00030780_000.png,00030780_000.png,train,1,0,0,0,0,0,0,0
19723,../input/data/images_012/images/00030786_000.png,00030786_000.png,train,0,0,1,0,0,0,0,0


In [16]:
from collections import namedtuple
import os
import torch
from torchvision import models
import torch.nn.functional as F
torch.manual_seed(98)

class AlignmentNetwork(torch.nn.Module):
    def __init__(self):
        super(AlignmentNetwork, self).__init__()

        resnet18 = models.resnet18()
        num_ftrs = resnet18.fc.in_features
        resnet18.fc = nn.Linear(num_ftrs, 6)

        self.model  = resnet18


    def forward(self, x):
        theta = self.model(x)
        theta = theta.view(-1, 2, 3)

        self.grid = F.affine_grid(theta, x.size())
        x = F.grid_sample(x, self.grid)
#         m = nn.AvgPool2d(16, stride=16)
#         x = m(x)
#         x = F.interpolate(x, (224,224), mode='bilinear')
        return x

In [ ]:
# the vgg16 is borrowed from the pytorch tutorial
# the resent is used to reimplemented and the comparision experiments
from collections import namedtuple
import os
import torch
from torchvision import models

class ResNet50(torch.nn.Module):
    def __init__(self):
        super(ResNet50, self).__init__()

        resnet50_pretrained = models.resnet50()
        num_ftrs = resnet50_pretrained.fc.in_features
        resnet50_pretrained.fc = nn.Sequential(
            nn.Linear(num_ftrs, 8), nn.Sigmoid())

        # resnet50_pretrained.load_state_dict(torch.load('/kaggle/input/models-for-localisation-testing/resnet50_5_chestxray8.pth'))

        self.model  = list(resnet50_pretrained.children())

        self.noslice1 = self.model[0]
        self.noslice2 = self.model[1]
        self.noslice3 = self.model[2]
        self.noslice4 = self.model[3]

        self.slice1 = self.model[4]
        self.slice2 = self.model[5]
        self.slice3 = self.model[6]
        self.slice4 = self.model[7]

#         self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
        self.conv = nn.Sequential(nn.Conv2d(in_channels=2048, out_channels=8, kernel_size=1))
                                # nn.ReLU(inplace=True))
        self.anothaone = nn.Sequential(nn.Conv2d(in_channels=8, out_channels=8, kernel_size=1))
                                # nn.ReLU(inplace=True))
        self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)), nn.Sigmoid())
        # self.fc = nn.Sequential(nn.AdaptiveAvgPool2d((1,1)),
        #                         nn.Flatten(),
        #                         nn.Linear(2048, 8),
        #                         nn.Sigmoid())

    def forward(self, x, alignment_network):
#         x = x.to(device)
        o = x
#         for i in range(4):
#             o = self.model[i](o)

        alignment_network.eval()
        o = alignment_network(o)
        aligned = o

        o = self.noslice1(o)
        o = self.noslice2(o)
        o = self.noslice3(o)
        o = self.noslice4(o)

        # o = (o * lung_mask) + o

#         anchor = cv.resize()
#         o = o*anchor
        o = self.slice1(o)
        feature1 = o
        # o = (o * lung_mask) + o
        o = self.slice2(o)
        feature2 = o
        o = self.slice3(o)
        feature3 = o
        o = self.slice4(o)
        feature4 = o
        o = self.conv(o)
        final = o
        if(o.shape[0] == 32):
            o = (o * batch_masks) + o
        else:
            temp = o.shape[0]
            o = (o * batch_masks[:temp]) + o
        o = self.anothaone(o)
        final2 = o
        o = self.fc(o)
        final3 = o

        return final3

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model = ResNet50()
model = model.to(device)
model = model.type(torch.cuda.FloatTensor)
model.load_state_dict(torch.load('/content/drive/MyDrive/final_models/no_align_5.pth'))
model.eval()
print('Model loaded')
# model.load_state_dict(torch.load('/content/drive/MyDrive/final_models/graddmask_new_align_10.pth'))

In [14]:
from torchvision import models
import torch.nn as nn

model = models.resnet50()
num_ftrs = model.fc.in_features
model.fc = nn.Sequential(
    nn.Linear(num_ftrs, 8), nn.Sigmoid())

model.load_state_dict(torch.load('/kaggle/input/models-for-localisation-testing/resnet50_5_chestxray8.pth', map_location=torch.device('cpu')))
model.eval()
print('Model loaded')

Model loaded


In [9]:
def large_to_small(x1, y1, w1, h1, cropped = True): # convert 1024x1024 to 224x224 (which is center-cropped from 256x256) 
    x2 = x1 / 4
    y2 = y1 / 4
    w2 = w1 / 4
    h2 = h1 / 4
    if cropped:
        if x2 < 16:
            x2 = 0
            w2 = w2 - 16
        else:
            x2 = x2 - 16
        if x2 + w2 > 224:
            w2 = 224 - x2
        if y2 < 16:
            y2 = 0
            h2 = h2 - 16
        else:
            y2 = y2 - 16
        if y2 + h2 > 224:
            h2 = 224 - y2
    return int(x2), int(y2), int(w2), int(h2)

def intersect(box_a, box_b):
    A = box_a.size(0)
    B = box_b.size(0)
    max_xy = torch.min(box_a[:, 2:].unsqueeze(1).expand(A, B, 2),
                       box_b[:, 2:].unsqueeze(0).expand(A, B, 2))
    min_xy = torch.max(box_a[:, :2].unsqueeze(1).expand(A, B, 2),
                       box_b[:, :2].unsqueeze(0).expand(A, B, 2))
    inter = torch.clamp((max_xy - min_xy), min=0)
    return inter[:, :, 0] * inter[:, :, 1]

def jaccard(box_a, box_b):
    inter = intersect(box_a, box_b)
    area_a = ((box_a[:, 2]-box_a[:, 0]) *
              (box_a[:, 3]-box_a[:, 1])).unsqueeze(1).expand_as(inter)  # [A,B]
    area_b = ((box_b[:, 2]-box_b[:, 0]) *
              (box_b[:, 3]-box_b[:, 1])).unsqueeze(0).expand_as(inter)  # [A,B]
    union = area_a + area_b - inter
    return inter / union  # [A,B]

def iobb(box_a, box_b):
    inter = intersect(box_a, box_b)
    area_a = ((box_a[:, 2]-box_a[:, 0]) *
              (box_a[:, 3]-box_a[:, 1])).unsqueeze(1).expand_as(inter)  # [A,B]
    area_b = ((box_b[:, 2]-box_b[:, 0]) *
              (box_b[:, 3]-box_b[:, 1])).unsqueeze(0).expand_as(inter)  # [A,B]
#     union = area_a + area_b - inter
    return inter / area_a  # [A,B]

def contains(xywh1, xywh2): # returns True if xywh2 is completely inside xywh1
    x1, x2, x3, x4 = xywh1
    y1, y2, y3, y4 = xywh2
    if y1 < x1:
        return False
    if y2 < x2:
        return False
    if y3 > x3:
        return False
    if y4 > x4:
        return False
    return True

In [10]:
import torchvision.transforms as T

preprocess = T.Compose([
   T.Resize(224),
   T.CenterCrop(224),
   T.ToTensor(),
   T.Normalize(
       mean=[0.485, 0.456, 0.406],
       std=[0.229, 0.224, 0.225]
   )
])

convert_tensor = T.Compose([
   T.ToTensor(),
])

In [42]:
if 2==5 or 2==3:
    print(True)

In [85]:
%%time

import io
import requests
import torchvision.transforms as T
import numpy as np
from PIL import Image

from torchvision import transforms
from torchvision.utils import draw_segmentation_masks
from torchvision.ops import masks_to_boxes
from torchvision.utils import draw_bounding_boxes
from matplotlib.patches import Rectangle

from skimage.filters import threshold_otsu

# cam_list = [GradCAM, GradCAMpp, LayerCAM]
cam_list = [LayerCAM]
iou_gcam, iou_gcampp, iou_lcam = [], [], []
contain_gcam, contain_gcampp, contain_lcam = [], [], []
pred_class = []
all_cams = []
prob_value = []
dis_0, dis_1, dis_2, dis_3, dis_4, dis_5, dis_6, dis_7 = [], [], [], [], [], [], [], []

thresh = 0.8

alignment_network = AlignmentNetwork()
alignment_network.load_state_dict(torch.load('/kaggle/input/final-testing/2feat_no_crop_fixed_pt_alignment_network_5.pth', map_location=torch.device('cpu')))
# alignment_network.load_state_dict(torch.load('/content/drive/MyDrive/fixed_pt_alignment_network_5.pth'))
# alignment_network = alignment_network.to(device)
alignment_network.eval()

for file in range(len(df)):
    if file % 500 == 0:
        print(file)
    for f in cam_list:
        temp_cams = []
#         cam_name = str(f).split('.')[3].split("'")[0]
        img_path = df['full_path'].iloc[file]
#         cam_extractor = f(model)
        img = Image.open(img_path)
        img = img.convert('RGB')
        img = preprocess(img)
        x  = alignment_network(img.unsqueeze(0))
        out = model(x)
        
#         out = out[0][:8].unsqueeze(0)
#         cams = cam_extractor(out.squeeze(0).argmax().item(), out)
#         cam_extractor.remove_hooks()
        for i in range(8):
            if (i==0):
                if(out.squeeze(0)[i] > thresh):
                    cam_extractor = f(model)
                    ind_out = model(img.unsqueeze(0))
                    ind_cam = cam_extractor(i, ind_out)
                    cam_extractor.remove_hooks()
                    dis_0.append(ind_cam)
                    
            elif (i==1):
                if(out.squeeze(0)[i] > thresh):
                    cam_extractor = f(model)
                    ind_out = model(img.unsqueeze(0))
                    ind_cam = cam_extractor(i, ind_out)
                    cam_extractor.remove_hooks()
                    dis_1.append(ind_cam)
                    
            elif (i==2):
                if(out.squeeze(0)[i] > thresh):
                    cam_extractor = f(model)
                    ind_out = model(img.unsqueeze(0))
                    ind_cam = cam_extractor(i, ind_out)
                    cam_extractor.remove_hooks()
                    dis_2.append(ind_cam)
                    
            elif (i==3):
                if(out.squeeze(0)[i] > (thresh - 0.1)):
                    cam_extractor = f(model)
                    ind_out = model(img.unsqueeze(0))
                    ind_cam = cam_extractor(i, ind_out)
                    cam_extractor.remove_hooks()
                    dis_3.append(ind_cam)
                    
            elif (i==4):
                if(out.squeeze(0)[i] > thresh):
                    cam_extractor = f(model)
                    ind_out = model(img.unsqueeze(0))
                    ind_cam = cam_extractor(i, ind_out)
                    cam_extractor.remove_hooks()
                    dis_4.append(ind_cam)
                    
            elif (i==5):
                if(out.squeeze(0)[i] > thresh):
                    cam_extractor = f(model)
                    ind_out = model(img.unsqueeze(0))
                    ind_cam = cam_extractor(i, ind_out)
                    cam_extractor.remove_hooks()
                    dis_5.append(ind_cam)

            elif (i==6):
                if(out.squeeze(0)[i] > (thresh - 0.3)) or (out.squeeze(0).argmax().item() == 6):
                    cam_extractor = f(model)
                    ind_out = model(img.unsqueze(0))
                    ind_cam = cam_extractor(i, ind_out)
                    cam_extractor.remove_hooks()
                    dis_6.append(ind_cam)
                    
            elif (i==7):
                if(out.squeeze(0)[i] > thresh):
                    cam_extractor = f(model)
                    ind_out = model(img.unsqueeze(0))
                    ind_cam = cam_extractor(i, ind_out)
                    cam_extractor.remove_hooks()
                    dis_7.append(ind_cam)

0
500
1000
1500
2000
2500
3000
3500
4000
4500
5000
5500
6000
6500
7000
7500
8000
8500
9000
9500
10000
10500
11000
11500
12000
12500
13000
13500
14000
14500
15000
15500
16000
16500
17000
17500
18000
18500
19000
19500
CPU times: user 1h 49min 51s, sys: 26.5 s, total: 1h 50min 17s
Wall time: 59min 43s


In [86]:
g_dis_0, g_dis_1, g_dis_2, g_dis_3, g_dis_4, g_dis_5, g_dis_6, g_dis_7 = dis_0, dis_1, dis_2, dis_3, dis_4, dis_5, dis_6, dis_7

In [87]:
a0, a1, a2, a3, a4, a5, a6, a7 = [], [], [], [], [], [], [], []
for i in g_dis_0:
    temp_i = np.flip(i[0][0].numpy(),1)
    a0.append(np.add(i[0][0], temp_i))
for i in g_dis_1:
    a1.append(i[0][0])
for i in g_dis_2:
    temp_i = np.flip(i[0][0].numpy(),1)
    a2.append(np.add(i[0][0], temp_i))
for i in g_dis_3:
    temp_i = np.flip(i[0][0].numpy(),1)
    a3.append(np.add(i[0][0], temp_i))
for i in g_dis_4:
    temp_i = np.flip(i[0][0].numpy(),1)
    a4.append(np.add(i[0][0], temp_i))
for i in g_dis_5:
    temp_i = np.flip(i[0][0].numpy(),1)
    a5.append(np.add(i[0][0], temp_i))
for i in g_dis_6:
    temp_i = np.flip(i[0][0].numpy(),1)
    a6.append(np.add(i[0][0], temp_i))
for i in g_dis_7:
    temp_i = np.flip(i[0][0].numpy(),1)
    a7.append(np.add(i[0][0], temp_i))

In [88]:
final_0 = np.zeros((7,7))
for i in a0:
    final_0 = np.add(final_0, i)
final_0 = final_0 / len(a0)
# final_0 = torch.Tensor(final_0).unsqueeze(0)

final_1 = np.zeros((7,7))
for i in a1:
    final_1 = np.add(final_1, i)
final_1 = final_1 / len(a1)
final_1 = torch.Tensor(final_1).unsqueeze(0)

final_2 = np.zeros((7,7))
for i in a2:
    final_2 = np.add(final_2, i)
final_2 = final_2 / len(a2)
final_2 = torch.Tensor(final_2).unsqueeze(0)

final_3 = np.zeros((7,7))
for i in a3:
    final_3 = np.add(final_3, i)
final_3 = final_3 / len(a3)
final_3 = torch.Tensor(final_3).unsqueeze(0)

final_4 = np.zeros((7,7))
for i in a4:
    final_4 = np.add(final_4, i)
final_4 = final_4 / len(a4)
final_4 = torch.Tensor(final_4).unsqueeze(0)

final_5 = np.zeros((7,7))
for i in a5:
    final_5 = np.add(final_5, i)
final_5 = final_5 / len(a5)
final_5 = torch.Tensor(final_5).unsqueeze(0)

final_6 = np.zeros((7,7))
for i in a6:
    final_6 = np.add(final_6, i)
final_6 = final_6 / len(a6)
final_6 = torch.Tensor(final_6).unsqueeze(0)

final_7 = np.zeros((7,7))
for i in a7:
    final_7 = np.add(final_7, i)
final_7 = final_7 / len(a7)
final_7 = torch.Tensor(final_7).unsqueeze(0)

In [89]:
final_0 = final_0.numpy()
final_1 = final_1.numpy()
final_2 = final_2.numpy()
final_3 = final_3.numpy()
final_4 = final_4.numpy()
final_5 = final_5.numpy()
final_6 = final_6.numpy()
final_7 = final_7.numpy()

final_0 = (final_0 - np.min(final_0))/np.ptp(final_0)
final_1 = (final_1 - np.min(final_1))/np.ptp(final_1)
final_2 = (final_2 - np.min(final_2))/np.ptp(final_2)
final_3 = (final_3 - np.min(final_3))/np.ptp(final_3)
final_4 = (final_4 - np.min(final_4))/np.ptp(final_4)
final_5 = (final_5 - np.min(final_5))/np.ptp(final_5)
final_6 = (final_6 - np.min(final_6))/np.ptp(final_6)
final_7 = (final_7 - np.min(final_7))/np.ptp(final_7)

final_0 = torch.Tensor(final_0)
final_1 = torch.Tensor(final_1)
final_2 = torch.Tensor(final_2)
final_3 = torch.Tensor(final_3)
final_4 = torch.Tensor(final_4)
final_5 = torch.Tensor(final_5)
final_6 = torch.Tensor(final_6)
final_7 = torch.Tensor(final_7)

In [90]:
final = [final_0, final_1, final_2, final_3, final_4, final_5, final_6, final_7]

In [ ]:
final[0].shape

In [96]:
np.save('final_fixed_aligned_layercam_normalized_masks.npy', np.array(final, dtype=object), allow_pickle=True)